<a href="https://colab.research.google.com/github/mahesh-ishran/Multi-Lingual-Sentiment-Analysis/blob/main/new_naive_without_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

multi_lingual_sentiment_analysis_path = kagglehub.competition_download('multi-lingual-sentiment-analysis')
metaresearch_llama_3_1_transformers_8b_instruct_2_path = kagglehub.model_download('metaresearch/llama-3.1/Transformers/8b-instruct/2')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/.gitattributes
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00002-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/gener

In [ ]:
# !pip install transformers peft datasets accelerate sentencepiece
# !pip install torch==2.6.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.1


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.1 MB/s eta 0:00:00:00:0100:01


In [ ]:
# !pip install torchvision

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer,BitsAndBytesConfig,pipeline
from datasets import load_dataset,DatasetDict,Dataset
from peft import LoraConfig, get_peft_model
from tqdm import tqdm

In [ ]:
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix,f1_score)
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


In [ ]:
data=pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/train.csv')

In [ ]:
data[data['language']=='bd']['sentence'][8]

'लेन्सा मैला हाबहोनो हायै रोखोमै आरो दागो होबथाहानाय महरै बानायजानाय नङा।'

In [ ]:
unique_languages_df = data.drop_duplicates(subset=['language'])
unique_languages_df

,ID,sentence,label,language
0,1,কর্মীদের ভাল আচরণ এবং খাবারের পাশাপাশি পানীয় ...,Positive,bn
1,2,ગોદરેજ સેન્ટ્રલ એસીમાં તેના કન્ડેન્સર પર 2 વર્...,Positive,gu
2,3,"கதைக்களம் பிடித்திருந்தது, அனைத்து நடிகர்களும்...",Positive,ta
3,4,ਵੌਇਸ-ਓਵਰ ਬਹੁਤ ਵਧੀਆ ਸੀ ਅਤੇ ਕਹਾਣੀ ਦੀ ਸੀਮਾ ਵਿੱਚ ਇ...,Positive,pa
4,5,जुथानि थाखाय जायगा गैया। गुबुन मुवा सोग्रा जाय...,Negative,bd
5,6,ইয়াত বহুতো জনপ্ৰিয় কিতাপ বিনামূলীয়াকৈ উপলব্...,Positive,as
6,7,సింఫనీ పర్సనల్ ఎయిర్ కూలర్ ఇప్పుడు హ్యుమిడిటీ ...,Positive,te
11,12,କ୍ଷୁଦ୍ର ସ୍ଥାନଗୁଡ଼ିକ ପାଇଁ ଏବଂ ସ୍ୱଳ୍ପ ଇଣ୍ଟରିୟର ପ...,Positive,or
12,13,"""ഈ വില ശ്രേണിയിൽ കസേരകൾക്ക് ക്രമീകരിക്കാവുന്ന ...",Positive,ml
13,14,"""चेयर कार में, अधिक चार्जिंग पॉइंट की आवश्यकता...",Negative,hi


In [ ]:
test=pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/test.csv')
test['language'].unique()

array(['pa', 'gu', 'kn', 'as', 'ur', 'te', 'ta', 'ml', 'bn', 'hi', 'or',
       'bd', 'mr'], dtype=object)

In [ ]:
lang_mapping = {
    'bn': 'Bangla',
    'gu': 'Gujarati',
    'ta': 'Tamil',
    'pa': 'Punjabi',
    'bd': 'Bodo',
    'as': 'Assamese',
    'te': 'Telugu',
    'or': 'Odia',
    'ml': 'Malayalam',
    'hi': 'Hindi',
    'mr': 'Marathi',
    'ur': 'Urdu',
    'kn': 'Kannada'
}

data['language'] = data['language'].replace(lang_mapping)

In [ ]:
data['language'].unique()

array(['Bangla', 'Gujarati', 'Tamil', 'Punjabi', 'Bodo', 'Assamese',
       'Telugu', 'Odia', 'Malayalam', 'Hindi', 'Marathi', 'Urdu',
       'Kannada'], dtype=object)

In [ ]:
df = data[['sentence','label','language']]

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)

print("Training set size:", train_df.shape)
print("Testing set size:", test_df.shape)

Training set size: (900, 3)
Testing set size: (100, 3)


In [ ]:
# Reset index for both train and test sets
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the sentence which is in language {data_point["language"]} enclosed in square brackets ,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label "Positive" or "Negative".
            [{data_point["sentence"]}] = {data_point["label"]}""".strip()

In [ ]:
x_train = pd.DataFrame(train_df.apply(generate_prompt, axis=1),
                       columns=["sentence"])
x_eval = pd.DataFrame(test_df.apply(generate_prompt, axis=1),
                      columns=["sentence"])

In [ ]:
x_train['sentence'][0]

'Analyze the sentiment of the sentence which is in language Gujarati enclosed in square brackets , \n            determine if it is positive or negative, and return the answer as \n            the corresponding sentiment label "Positive" or "Negative".\n            [વોલ્ટાસ સેન્ટ્રલ એસી જે સંયુક્ત બાષ્પીભવક સાથે આવે છે તે વધુ ઉર્જા કાર્યક્ષમ છે કારણ કે તે ઠંડક અને ગરમી બંને પ્રક્રિયાઓ માટે એક એકમનો ઉપયોગ કરે છે.] = Positive'

In [ ]:
train_data = Dataset.from_pandas(x_train)
eval_data = Dataset.from_pandas(x_eval)

In [ ]:
train_data

Dataset({
    features: ['sentence'],
    num_rows: 900
})

In [ ]:
eval_data

Dataset({
    features: ['sentence'],
    num_rows: 100
})

#### x_test

In [ ]:
k = 100  # Number of rows you want
random_rows = df.sample(n=k, random_state=42)  # Set random_state for reproducibility

x_test=random_rows
x_test = x_test.reset_index(drop=True)

In [ ]:
x_test[:2]

,sentence,label,language
0,4-6 ವಯಸ್ಸಿನವರಿಗೆ ತುಂಬಾ ಭಾರವಾಗುತ್ತದೆ. ಮಕ್ಕಳು ಸ್...,Negative,Kannada
1,ওরিও স্যান্ডউইচ বিস্কুটে ক্রিম ফিলিং এত সমৃদ্ধ...,Positive,Bangla


In [ ]:
def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the sentence which is in language {data_point["language"]} enclosed in square brackets ,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label "Positive" or "Negative".
            [{data_point["sentence"]}] = """.strip()

In [ ]:
y_true = x_test.label
x_test = pd.DataFrame(x_test.apply(generate_test_prompt, axis=1), columns=["sentence"])

In [ ]:
def evaluate(y_true, y_pred):
    labels = ['Positive', 'Negative']
    mapping = {'Positive': 1, 'Negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Calculate F1-score
    f1 = f1_score(y_true, y_pred)
    print(f'F1 Score: {f1:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

## without Fine-Tuning

### Model

In [ ]:
model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"

In [ ]:

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True
)

# Reload the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
)
print('compute dtype : ',compute_dtype)
print("Model successfully quantized to 4-bit!")
print(10*'_',f"Base Model size: {model.get_memory_footprint():,} bytes\n")

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

compute dtype :  torch.float16
Model successfully quantized to 4-bit!
__________ Base Model size: 5,591,548,160 bytes



### Tokenizer

In [ ]:
# Load tokenizer
max_seq_length = 512
tokenizer = AutoTokenizer.from_pretrained(model_name,max_seq_length=max_seq_length)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Function to calculate max token length
def get_max_token_length(dataset):
    max_length = 0
    for sentence in dataset['sentence']:
        tokenized = tokenizer(sentence)
        max_length = max(max_length, len(tokenized['input_ids']))  # Length of tokenized sentence
    return max_length

# Calculate max token length for both train and validation datasets
max_train_length = get_max_token_length(train_data)
max_val_length = get_max_token_length(eval_data)

print(max_train_length)
print(max_val_length)

807
1106


In [ ]:
import sys
def predict(x_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(x_test)),dynamic_ncols=True, desc="Processing"):
        if i == 0:  # Show this message only once, or suppress it entirely
            sys.stdout.write("Device set to use cuda:0\n")
        prompt = x_test.iloc[i]["sentence"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 1,
                        temperature=0.1)
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "Positive" in answer:
            y_pred.append("Positive")
        elif "Negative" in answer:
            y_pred.append("Negative")
        else:
            y_pred.append("Positive")
    return y_pred

In [ ]:
y_pred = predict(x_test, model, tokenizer)

In [ ]:
evaluate(y_true, y_pred)

Accuracy: 0.750
F1 Score: 0.809
Accuracy for label 0: 0.564
Accuracy for label 1: 0.869

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.56      0.64        39
           1       0.76      0.87      0.81        61

    accuracy                           0.75       100
   macro avg       0.75      0.72      0.72       100
weighted avg       0.75      0.75      0.74       100


Confusion Matrix:
[[22 17]
 [ 8 53]]


In [ ]:
# Apply LoRA for memory-efficient fine-tuning
# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     target_modules=["q_proj", "v_proj"]
# )

# model = get_peft_model(model, lora_config)
# model.print_trainable_parameters()

In [ ]:
test = pd.DataFrame(test.apply(generate_test_prompt, axis=1), columns=["sentence"])

In [ ]:
y_pred_test=predict(test, model, tokenizer)

Processing:   0%|          | 0/100 [00:00<?, ?it/s]Device set to use cuda:0


Device set to use cuda:0


Processing: 100%|██████████| 100/100 [00:40<00:00,  2.50it/s]


In [ ]:
test_ids = pd.read_csv("/kaggle/input/multi-lingual-sentiment-analysis/test.csv")["ID"]
# Create submission file with index
submission_df = pd.DataFrame({'ID': test_ids, 'label': y_pred_test})

# Save as CSV
submission_df.to_csv("submission.csv", index=False)